In [1]:
# file to compare number of queries per pcap file


# chrony-based VM



## 1. VM Setup

```shell

$ multipass launch --name my-vm
$ multipass shell my-vm

$lsb_release -a
No LSB modules are available.
Distributor ID: Ubuntu
Description:    Ubuntu 24.04.3 LTS
Release:        24.04
Codename:       noble


$ sudo apt install chrony

```


## 2.  inspecting chrony configuration

```shell

$ cat /etc/chrony/
$ cat /etc/chrony/chrony.conf

# Welcome to the chrony configuration file. See chrony.conf(5) for more
# information about usable directives.

# Include configuration files found in /etc/chrony/conf.d.
confdir /etc/chrony/conf.d

# This will use (up to):
# - 4 sources from ntp.ubuntu.com which some are ipv6 enabled
# - 2 sources from 2.ubuntu.pool.ntp.org which is ipv6 enabled as well
# - 1 source from [01].ubuntu.pool.ntp.org each (ipv4 only atm)
# This means by default, up to 6 dual-stack and up to 2 additional IPv4-only
# sources will be used.
# At the same time it retains some protection against one of the entries being
# down (compare to just using one of the lines). See (LP: #1754358) for the
# discussion.
#
# About using servers from the NTP Pool Project in general see (LP: #104525).
# Approved by Ubuntu Technical Board on 2011-02-08.
# See http://www.pool.ntp.org/join.html for more information.
pool ntp.ubuntu.com        iburst maxsources 4
pool 0.ubuntu.pool.ntp.org iburst maxsources 1
pool 1.ubuntu.pool.ntp.org iburst maxsources 1
pool 2.ubuntu.pool.ntp.org iburst maxsources 2

# Use time sources from DHCP.
sourcedir /run/chrony-dhcp

# Use NTP sources found in /etc/chrony/sources.d.
sourcedir /etc/chrony/sources.d

# This directive specify the location of the file containing ID/key pairs for
# NTP authentication.
keyfile /etc/chrony/chrony.keys

# This directive specify the file into which chronyd will store the rate
# information.
driftfile /var/lib/chrony/chrony.drift

# Save NTS keys and cookies.
ntsdumpdir /var/lib/chrony

# Uncomment the following line to turn logging on.
#log tracking measurements statistics

# Log files location.
logdir /var/log/chrony

# Stop bad estimates upsetting machine clock.
maxupdateskew 100.0

# This directive enables kernel synchronisation (every 11 minutes) of the
# real-time clock. Note that it can't be used along with the 'rtcfile' directive.
rtcsync

# Step the system clock instead of slewing it if the adjustment is larger than
# one second, but only in the first three clock updates.
makestep 1 3

# Get TAI-UTC offset and leap seconds from the system tz database.
# This directive must be commented out when using time sources serving
# leap-smeared time.
leapsectz right/UTC
````

## chrony status

* Not sure why I have these other NTP servers whihc are not canonical -- are they part of Canonical's new pool?
  * I have no `/run/chrony-dhcp` file
  * I have no  servers under ` /etc/chrony/sources.d` , only a `README` file
  * Oh wait, my version is using NTP POOL


```shell
 chronyc sources

MS Name/IP address         Stratum Poll Reach LastRx Last sample               
===============================================================================
^+ prod-ntp-5.ntp1.ps5.cano>     2   9   377   120   +518us[ +518us] +/- 6635us
^+ prod-ntp-3.ntp4.ps5.cano>     2  10   377    93   +872us[ +872us] +/- 8139us
^- alphyn.canonical.com          2   9   377   106   -429us[ -429us] +/-   62ms
^+ prod-ntp-4.ntp4.ps5.cano>     2  10   377   254   +416us[ +416us] +/- 7265us
^- mail.roundowl.tk              2  10   377   514   -690us[ -690us] +/-   13ms
^- ntp2.as215248.net             2  10   377     3    +36ms[  +36ms] +/-  546ms
^* connected.by.freedominte>     2  10   377   964   -412us[ -560us] +/- 7317us
^+ ntp.brodgar.net               1  10   377   559   -563us[ -563us] +/-   11ms
ubuntu@my-vm:~$ 

$ chronyc -n sources 
MS Name/IP address         Stratum Poll Reach LastRx Last sample               
===============================================================================
^+ 185.125.190.58                2   9   377   191   +338us[ +518us] +/- 6635us
^+ 185.125.190.56                2  10   377   164   +692us[ +872us] +/- 8139us
^- 91.189.91.157                 2   9   377   177   -609us[ -429us] +/-   62ms
^+ 185.125.190.57                2  10   377   325   +235us[ +416us] +/- 7265us
^- 5.255.99.180                  2  10   377   585   -871us[ -690us] +/-   13ms
^- 103.149.168.181               2  10   377    74    +36ms[  +36ms] +/-  546ms
^* 178.239.19.62                 2  10   377     5   -562us[ -382us] +/- 6862us
^+ 93.115.17.154                 1  10   377   630   -744us[ -563us] +/-   11ms


^ = server
* = currently selected source for synchronization
+ = acceptable source
- = source not used
? = unreachable or not yet reachable
x = source with a false tick
~ = source with a variable rate

```

# timesyncd setup


## 1. VM Setup

```shell

$ multipass launch --name timesyncd
$ multipass shell my-vm

$ lsb_release -a
No LSB modules are available.
Distributor ID: Ubuntu
Description:    Ubuntu 24.04.3 LTS
Release:        24.04
Codename:       noble
```





## 2.  inspecting timesyncd configuration

``` shell 

cat  /etc/systemd/timesyncd.conf 
#  This file is part of systemd.
#
#  systemd is free software; you can redistribute it and/or modify it under the
#  terms of the GNU Lesser General Public License as published by the Free
#  Software Foundation; either version 2.1 of the License, or (at your option)
#  any later version.
#
# Entries in this file show the compile time defaults. Local configuration
# should be created by either modifying this file (or a copy of it placed in
# /etc/ if the original file is shipped in /usr/), or by creating "drop-ins" in
# the /etc/systemd/timesyncd.conf.d/ directory. The latter is generally
# recommended. Defaults can be restored by simply deleting the main
# configuration file and all drop-ins located in /etc/.
#
# Use 'systemd-analyze cat-config systemd/timesyncd.conf' to display the full config.
#
# See timesyncd.conf(5) for details.

[Time]
#NTP=
#FallbackNTP=ntp.ubuntu.com
#RootDistanceMaxSec=5
#PollIntervalMinSec=32
#PollIntervalMaxSec=2048
#ConnectionRetrySec=30
#SaveIntervalSec=60
```

## timesyncd statuts

```shell

               Local time: Tue 2025-09-09 13:40:57 CEST
           Universal time: Tue 2025-09-09 11:40:57 UTC
                 RTC time: Tue 2025-09-09 11:40:57
                Time zone: Europe/Amsterdam (CEST, +0200)
System clock synchronized: yes
              NTP service: active
          RTC in local TZ: no

```

# Source code to compare pcaps

In [4]:

# function to parse pcap files captured from VMs

from scapy.all import rdpcap, UDP
from collections import defaultdict


# Reads a PCAP file and analyzes NTP traffic, summarizing per client-server pair:
# number of queries/responses, total sizes, and time between first and last query.
def analyze_ntp_pcap(pcap_file):
    packets = rdpcap(pcap_file)
    ntp_data = defaultdict(lambda: {
        'number_of_queries': 0,
        'number_of_responses': 0,
        'size_of_queries': 0,
        'size_of_responses': 0,
        'first_query_time': None,
        'last_query_time': None
    })

    for pkt in packets:
        if pkt.haslayer(UDP) and (pkt[UDP].sport == 123 or pkt[UDP].dport == 123):
            src_ip = pkt[0][1].src
            dst_ip = pkt[0][1].dst
            length = len(pkt)
            timestamp = pkt.time

            if pkt[UDP].dport == 123:  # NTP query
                key = (src_ip, dst_ip)
                ntp_data[key]['number_of_queries'] += 1
                ntp_data[key]['size_of_queries'] += length
                if ntp_data[key]['first_query_time'] is None:
                    ntp_data[key]['first_query_time'] = timestamp
                ntp_data[key]['last_query_time'] = timestamp
            elif pkt[UDP].sport == 123:  # NTP response
                key = (dst_ip, src_ip)
                ntp_data[key]['number_of_responses'] += 1
                ntp_data[key]['size_of_responses'] += length

    # Final formatting and sorting
    result = []
    for (client, server), data in ntp_data.items():
        time_diff = None
        queries_per_hour = None
        responses_per_hour = None

        if data['first_query_time'] and data['last_query_time']:
            time_diff = data['last_query_time'] - data['first_query_time']
            hours = time_diff / 3600 if time_diff > 0 else 1  # avoid division by zero
            queries_per_hour = data['number_of_queries'] / hours
            responses_per_hour = data['number_of_responses'] / hours

        result.append({
            'client': client,
            'ntp_server_ip': server,
            'number_of_queries': data['number_of_queries'],
            'number_of_responses': data['number_of_responses'],
            'size_of_queries': data['size_of_queries'],
            'size_of_responses': data['size_of_responses'],
            'time_between_first_and_last_query': time_diff,
            'queries_per_hour': queries_per_hour,
            'responses_per_hour': responses_per_hour
        })

    # Sort by number_of_queries descending
    result.sort(key=lambda x: x['number_of_queries'], reverse=True)

    return result



# Prints the NTP summary data as a Markdown-formatted table, sorted by number of queries in descending order.
# Adds a "Total" row that sums all queries, responses, sizes, and computes average query/response rates per hour.

def print_ntp_summary_markdown(summary_data):
    # Sort by number_of_queries descending
    sorted_data = sorted(summary_data, key=lambda x: x['number_of_queries'], reverse=True)

    print("| Client IP | NTP Server IP | # Queries | # Responses | Query Size (bytes) | Response Size (bytes) | Time Span (s) | Queries/hr | Responses/hr |")
    print("|-----------|----------------|-----------|--------------|--------------------|------------------------|---------------|------------|--------------|")

    total_queries = 0
    total_responses = 0
    total_query_size = 0
    total_response_size = 0
    total_time_span = 0
    valid_time_spans = 0  # count of entries with valid time spans

    for entry in sorted_data:
        time_span = entry['time_between_first_and_last_query']
        time_span_hours = time_span / 3600 if time_span else None
        queries_per_hour = entry['number_of_queries'] / time_span_hours if time_span_hours else 0
        responses_per_hour = entry['number_of_responses'] / time_span_hours if time_span_hours else 0

        time_diff_str = f"{time_span:.2f}" if time_span else "N/A"
        qph_str = f"{queries_per_hour:.2f}" if time_span_hours else "N/A"
        rph_str = f"{responses_per_hour:.2f}" if time_span_hours else "N/A"

        print(f"| {entry['client']} | {entry['ntp_server_ip']} | {entry['number_of_queries']} | {entry['number_of_responses']} | {entry['size_of_queries']} | {entry['size_of_responses']} | {time_diff_str} | {qph_str} | {rph_str} |")

        # Accumulate totals
        total_queries += entry['number_of_queries']
        total_responses += entry['number_of_responses']
        total_query_size += entry['size_of_queries']
        total_response_size += entry['size_of_responses']
        if time_span:
            total_time_span += time_span
            valid_time_spans += 1

    # Compute average time span and rates
    avg_time_span_hours = (total_time_span / valid_time_spans) / 3600 if valid_time_spans else None
    total_qph = total_queries / avg_time_span_hours if avg_time_span_hours else 0
    total_rph = total_responses / avg_time_span_hours if avg_time_span_hours else 0
    avg_time_span_str = f"{total_time_span:.2f}" if valid_time_spans else "N/A"
    total_qph_str = f"{total_qph:.2f}" if avg_time_span_hours else "N/A"
    total_rph_str = f"{total_rph:.2f}" if avg_time_span_hours else "N/A"

    # Print total line
    print(f"| **Total** | - | {total_queries} | {total_responses} | {total_query_size} | {total_response_size} | {avg_time_span_str} | {total_qph_str} | {total_rph_str} |")


# Results

* I captured traffic on both VMs
* We compare them

In [5]:
#first, timesyncd

data=analyze_ntp_pcap('timesyncd.pcap')
print_ntp_summary_markdown(data)

| Client IP | NTP Server IP | # Queries | # Responses | Query Size (bytes) | Response Size (bytes) | Time Span (s) | Queries/hr | Responses/hr |
|-----------|----------------|-----------|--------------|--------------------|------------------------|---------------|------------|--------------|
| 10.35.191.133 | 185.125.190.56 | 13 | 13 | 1170 | 1170 | 4995.25 | 9.37 | 9.37 |
| **Total** | - | 13 | 13 | 1170 | 1170 | 4995.25 | 9.37 | 9.37 |


* I just copied the previous output here to render as markdown

| Client IP | NTP Server IP | # Queries | # Responses | Query Size (bytes) | Response Size (bytes) | Time Span (s) | Queries/hr | Responses/hr |
|-----------|----------------|-----------|--------------|--------------------|------------------------|---------------|------------|--------------|
| 10.35.191.133 | 185.125.190.56 | 13 | 13 | 1170 | 1170 | 4995.25 | 9.37 | 9.37 |
| **Total** | - | 13 | 13 | 1170 | 1170 | 4995.25 | 9.37 | 9.37 |

In [6]:
# second, chrony

data2=analyze_ntp_pcap('chrony.pcap')
print_ntp_summary_markdown(data2)

| Client IP | NTP Server IP | # Queries | # Responses | Query Size (bytes) | Response Size (bytes) | Time Span (s) | Queries/hr | Responses/hr |
|-----------|----------------|-----------|--------------|--------------------|------------------------|---------------|------------|--------------|
| 10.35.191.89 | 185.125.190.58 | 56 | 56 | 5040 | 5040 | 5685.63 | 35.46 | 35.46 |
| 10.35.191.89 | 93.115.17.154 | 49 | 49 | 4410 | 4410 | 5762.29 | 30.61 | 30.61 |
| 10.35.191.89 | 185.125.190.57 | 48 | 48 | 4320 | 4320 | 5560.10 | 31.08 | 31.08 |
| 10.35.191.89 | 103.149.168.181 | 45 | 45 | 4050 | 4050 | 5812.63 | 27.87 | 27.87 |
| 10.35.191.89 | 178.239.19.62 | 40 | 40 | 3600 | 3600 | 5373.45 | 26.80 | 26.80 |
| 10.35.191.89 | 91.189.91.157 | 36 | 36 | 3240 | 3240 | 5685.78 | 22.79 | 22.79 |
| 10.35.191.89 | 5.255.99.180 | 34 | 34 | 3060 | 3060 | 5805.82 | 21.08 | 21.08 |
| 10.35.191.89 | 185.125.190.56 | 28 | 28 | 2520 | 2520 | 5182.80 | 19.45 | 19.45 |
| **Total** | - | 336 | 336 | 30240 | 3

* output copied here for readabitliy


| Client IP | NTP Server IP | # Queries | # Responses | Query Size (bytes) | Response Size (bytes) | Time Span (s) | Queries/hr | Responses/hr |
|-----------|----------------|-----------|--------------|--------------------|------------------------|---------------|------------|--------------|
| 10.35.191.89 | 185.125.190.58 | 56 | 56 | 5040 | 5040 | 5685.63 | 35.46 | 35.46 |
| 10.35.191.89 | 93.115.17.154 | 49 | 49 | 4410 | 4410 | 5762.29 | 30.61 | 30.61 |
| 10.35.191.89 | 185.125.190.57 | 48 | 48 | 4320 | 4320 | 5560.10 | 31.08 | 31.08 |
| 10.35.191.89 | 103.149.168.181 | 45 | 45 | 4050 | 4050 | 5812.63 | 27.87 | 27.87 |
| 10.35.191.89 | 178.239.19.62 | 40 | 40 | 3600 | 3600 | 5373.45 | 26.80 | 26.80 |
| 10.35.191.89 | 91.189.91.157 | 36 | 36 | 3240 | 3240 | 5685.78 | 22.79 | 22.79 |
| 10.35.191.89 | 5.255.99.180 | 34 | 34 | 3060 | 3060 | 5805.82 | 21.08 | 21.08 |
| 10.35.191.89 | 185.125.190.56 | 28 | 28 | 2520 | 2520 | 5182.80 | 19.45 | 19.45 |
| **Total** | - | 336 | 336 | 30240 | 30240 | 44868.49 | 215.67 | 215.67 |

## Comparison

* `timesyncd` uses only 1 IP address and sends 10 queries per hour in this 49925s (1.4h) experiment
* `chrony` uses *8* IP addresses and sends between 19 and 45 queries per hour per sever -- in total 215 per hour, in this 1.6h experiment
   * the difference is a 20x traffic growth